# Fourier Neural Operator

In [1]:
import neuralop
import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import os

torch.manual_seed(2002)
np.random.seed(2002)

## Постановка задачи

В качестве входных данных сеть получает начальное условие для уравнения Бюргерса $$ {\frac {\partial u}{\partial t}}+u{\frac {\partial u}{\partial x}}=\nu {\frac {\partial ^{2}u}{\partial x^{2}}}$$ на интервале $[0, length / 2]$ и экстраполирует его на интервал $[length / 2, length]$, где length=256.

## Этап 1: обучение FNO из пакета neuralop

На этом этапе задача будет решаться с официальной реализацией FNO из пакета neuralop.

### Загрузка и обработка данных

Путь к данным. Лучше поменять, так как я прописывал его относительно базовой директории окружения.

In [2]:
datasets_path = "./Task2_with_datasets/datasets"

In [3]:
from dataloaders.burgers import load_burgers_1dtime
from dataloaders.tensor_dataset import TensorDataset

При загрузке данным надо вытащить сами массивы данных, чтобы разбить их на x и y.

In [4]:
_, _, _, train, test = load_burgers_1dtime(
    data_path=os.path.join(datasets_path, "burgers"),
    n_train=400, n_test=100, batch_size=16, batch_size_test=16,
    temporal_length=1, spatial_length=256)

In [5]:
print(train.shape, test.shape)

torch.Size([400, 3, 1, 256]) torch.Size([100, 3, 1, 256])


In [6]:
train = torch.squeeze(train, dim=2)
test = torch.squeeze(test, dim=2)
print(train.shape, test.shape)

torch.Size([400, 3, 256]) torch.Size([100, 3, 256])


In [7]:
x_train = train[:, :, :128]
y_train =  torch.unsqueeze(train[:, 0, 128:], dim=1)

x_test = test[:, :, :128]
y_test = torch.unsqueeze(test[:, 0, 128:], dim=1)
train_db = TensorDataset(x_train, y_train)
train_loader = torch.utils.data.DataLoader(train_db, batch_size=32, shuffle=False)

test_db = TensorDataset(x_test, y_test)
test_loader = torch.utils.data.DataLoader(test_db, batch_size=16, shuffle=False)

### Создание модели

In [49]:
from neuralop.models import FNO
from neuralop.utils import count_params
from neuralop import LpLoss, H1Loss
import sys

Создадим 1-d FNO:

In [50]:
device = "cuda"

model = FNO(n_modes=(32,), in_channels=3, out_channels=1, n_layers=6, hidden_channels=32, projection_channels=64)
model = model.to(device)

n_params = count_params(model)
print(f'\nOur model has {n_params} parameters.')
sys.stdout.flush()

optimizer = torch.optim.Adam(model.parameters(),
                             lr=1e-3)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100000, gamma=1)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=5e-4)

h1loss = H1Loss(d=1, reductions='mean')
l2loss =LpLoss(d=1, p=2, reductions='mean')

train_loss = h1loss
eval_losses={'h1': h1loss, 'l2': l2loss}

print('\n### MODEL ###\n', model)
print('\n### OPTIMIZER ###\n', optimizer)
print('\n### SCHEDULER ###\n', scheduler)
print('\n### LOSSES ###')
print(f'\n * Train: {train_loss}')
print(f'\n * Test: {eval_losses}')
sys.stdout.flush()


Our model has 205249 parameters.

### MODEL ###
 FNO(
  (fno_blocks): FNOBlocks(
    (convs): FactorizedSpectralConv(
      (weight): ModuleList(
        (0-5): 6 x ComplexDenseTensor(shape=torch.Size([32, 32, 16]), rank=None)
      )
    )
    (fno_skips): ModuleList(
      (0-5): 6 x Conv1d(32, 32, kernel_size=(1,), stride=(1,), bias=False)
    )
  )
  (lifting): Lifting(
    (fc): Conv1d(3, 32, kernel_size=(1,), stride=(1,))
  )
  (projection): Projection(
    (fc1): Conv1d(32, 64, kernel_size=(1,), stride=(1,))
    (fc2): Conv1d(64, 1, kernel_size=(1,), stride=(1,))
  )
)

### OPTIMIZER ###
 Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    initial_lr: 0.001
    lr: 0.001
    maximize: False
    weight_decay: 0
)

### SCHEDULER ###

### LOSSES ###

 * Train: <neuralop.training.losses.H1Loss object at 0x7fecb26f59a0>

 * Test: {'h1': <neuralop.training.losses.H1Los

Создаем Trainer с коллбэком для Tensorboard и обучаем модель:

In [51]:
from callbacks.tensorboard_callback import TensorBoardCallback
from callbacks.trainer import Trainer

In [52]:
exp_num = 4

In [53]:
tb_callback = TensorBoardCallback(log_dir=f'./logs/run{exp_num}')
trainer = Trainer(model=model, n_epochs=202,
                  device=device,
                  wandb_log=False,
                  log_test_interval=3,
                  use_distributed=False,
                  verbose=True,
                  callbacks=[tb_callback])

trainer.train(train_loader=train_loader, model=model, 
              output_encoder=None,
              test_loaders=test_loader,
              optimizer=optimizer,
              scheduler=scheduler,
              regularizer=False,
              training_loss=train_loss,
              eval_losses=eval_losses)

using standard method to load data to device.
using standard method to compute loss.
Training on regular inputs (no multi-grid patching).
Training on 400 samples
Testing on [100] samples         on resolutions ['test'].
Training on raw inputs of size x.shape=torch.Size([32, 3, 128]), y.shape=torch.Size([32, 1, 128])
.. patched inputs of size x.shape=torch.Size([32, 3, 128]), y.shape=torch.Size([32, 1, 128])
Raw outputs of size out.shape=torch.Size([32, 1, 128])
.. Processed (unpatched) outputs of size out.shape=torch.Size([32, 1, 128])
[0] time=0.12, avg_loss=0.0647, train_err=0.0327, test_h1=0.0702, test_l2=0.0699
[3] time=0.12, avg_loss=0.0577, train_err=0.0291, test_h1=0.0588, test_l2=0.0591
[6] time=0.12, avg_loss=0.0527, train_err=0.0266, test_h1=0.0525, test_l2=0.0580
[9] time=0.12, avg_loss=0.0499, train_err=0.0252, test_h1=0.0497, test_l2=0.0569
[12] time=0.12, avg_loss=0.0426, train_err=0.0215, test_h1=0.0408, test_l2=0.0421
[15] time=0.12, avg_loss=0.0395, train_err=0.0200, t

Из результатов экспериментов -- MSE на тестовой выборке после послежней эпохи в районе $4 \cdot 10^{-2}$. Кажется, это неплохой результат.

Посмотрим на результаты экспериментов в Tensorboard.

In [46]:
!tensorboard --logdir=logs

2023-12-01 13:00:15.540470: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-01 13:00:15.964079: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-12-01 13:00:16.514837: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-01 13:00:16.515284: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n